In [99]:
import torch
import torch.nn as nn
import numpy as np
from torch.nn import functional as F
device = 'cuda' if torch .cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000

learning_rate = 3e-4
eval_iters = 250

cuda


In [100]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [101]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)

In [102]:
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [103]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[67, 57,  1, 73, 61, 54, 73,  1],
        [54, 72, 72, 58, 57,  1, 73, 61],
        [73, 73, 58, 67, 57, 54, 67, 73],
        [54, 73, 61, 72,  1, 73, 61, 71]], device='cuda:0')
targets:
tensor([[57,  1, 73, 61, 54, 73,  1, 58],
        [72, 72, 58, 57,  1, 73, 61, 71],
        [73, 58, 67, 57, 54, 67, 73, 72],
        [73, 61, 72,  1, 73, 61, 71, 68]], device='cuda:0')


In [104]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [105]:
class bigram_argument_model(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if  targets is None:
            loss = None
        else:
            B,T,C =  logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)

        return logits,loss
    
    def genrate(self,index, max_new_token):
        for _ in range(max_new_token):
            logits, loss = self.forward(index)
            logits = logits[:,-1,:]

            prob = F.softmax(logits,dim=-1)

            index_next = torch.multinomial(prob, num_samples=1)
            index = torch.cat((index,index_next),  dim=-1)
        
        return index
    
model = bigram_argument_model(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long,  device=device)
genrated_chars = decode(m.genrate(context,max_new_token=500)[0].tolist())
print(genrated_chars)


00x:i6ecHN:Uk7﻿swK'OjD6ce6XsO9
xRz)Knj:G3M7hzyB18(UNWj.4]R6nNB:U*Q00BkYdXlozv-)W1"BV.;MliZnvQ4s_Ml3;qNLhSUFuv"DA[9Ic6(;"q;!MJ;g9pZd[w'Sy_
ANR6j96g[x*;NV.4
EeUZ&rP!hzovHX(*T&22CM]UE;q:;Hn?jbl&Q;8(
*35_y  bAM7'5)v)r)a.?Ds.aRAnN:M-.? tTDeli3EEH"ZkI;BdLIW)aTVBNL[VA]-n8IPG1"[PsOjsvCPi&
XsjS"ye)CMlYCcZaw*U9B(-LU"kn oRQXTMBz'5gW,G*Y;]lFtLPi3T:U9eiPX9q)Uecr; Hw[y:
R_00'0
qPuGRN2yoM-6X
*Y;j
"9e1(MTppcDP.Vp_ZnrFNg!DA]-0y1Zov
'qh!L25rX5"w*Yceh!,2kO1)c5]SN?;*2U864'X;AvGrP0-au)6ti9]s﻿aFnURf﻿)cdma)IdouAZS :lh


In [106]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())  

step: 0, train loss: 4.875, val loss: 4.912
step: 250, train loss: 4.825, val loss: 4.840
step: 500, train loss: 4.774, val loss: 4.776
step: 750, train loss: 4.734, val loss: 4.715
4.768762111663818


In [107]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.genrate(context, max_new_token=500)[0].tolist())
print(generated_chars)


UzO9cU6AMku(0-)3*?"h﻿sjG-frz_KHR6j.P9w*69el43o6U1JS
&222!HYoRzc,Q﻿SaW *2!G*Q,tHk1VuU*QJ﻿WkjG
&
'0EHLG9ds._C-a4jG4Yf-(oW
w]-Av[
*1VOuZ0nbGv,t SfeTTNfyikvqr [nP7_ [OT3QNLCP,G﻿qdwll*CXU
]1m*(BmfXvj2LhLU1:hG[h7UWTXFN:2!E3E:)erB6XVSl[XHF(8G)*C3CCn:Ku&EO9wr[KZ[Unt3﻿q&﻿C;Bi6t.?X*5_F22P7(
B3&*8C8"y_,p4W?)Yq)lakB"9EHkV5_ m&2P'o,9.4YzcYH!gI;_i[G66Ka)rCYiWH '*UkGEDrKH96'[elzc6XsuXcx9f
hisj﻿wWenrqi3ns"﻿)h9emat.4sJXc16eG_vlPDWBusiLIWzX5-As.m6esdCFR5Uu&.h!*Ou:7,Frwr_ (;
DHN&fTgr9Hg
H_nbZqdI4&
;d7bbG)5 7,EkDBZ
